In [31]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
import numpy as np 
import pandas as pd 
import re
import nltk
from preprocess import *
# from models import *

In [32]:
df = pd.read_csv("questions.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [33]:
# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()
# is_duplicate = df['is_duplicate'].to_list()
# preprocess_neural(question_1, question_2, is_duplicate)

In [34]:
df1 = pd.read_csv("preprocessed_neural.csv")
q1_preprocessed, q2_preprocessed, is_duplicate = df1['question1'].to_list(), df1['question2'].to_list(), df1['is_duplicate'].to_list()

In [35]:
# change all 0 to 0 1 and 1 to 1 0
for i in range(len(is_duplicate)):
    if is_duplicate[i] == 0:
        is_duplicate[i] = [0,1]
    else:
        is_duplicate[i] = [1,0]

Acquired Test data

In [36]:
MAX_WORDS_VOCAB = 200000
tokenizer = Tokenizer(num_words = MAX_WORDS_VOCAB)
tokenizer.fit_on_texts(list(df1['question1'].values.astype(str))+list(df1['question2'].values.astype(str)))

In [37]:

q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
q1_sequence = pad_sequences(q1_sequence, maxlen = 30, padding='post')

q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
q2_sequence = pad_sequences(q2_sequence, maxlen = 30, padding='post')

In [38]:
windex = tokenizer.word_index

In [39]:
embedding_index = {}
with open('glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [40]:
embedding_matrix = np.random.random((len(windex)+1, 300))

for word, i in windex.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(108101, 300)


In [41]:
#split the data into 70-20-10 train-validation-test with random state 42
from sklearn.model_selection import train_test_split
q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_sequence, q2_sequence, is_duplicate, test_size=0.1, random_state=42)
q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.2, random_state=42)

In [42]:
print(q1_train.shape, q1_test.shape, q1_val.shape)

(291132, 30) (40436, 30) (72783, 30)


In [43]:
#print the ratio of positive and negative samples in train, validation and test
y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)
print("Train: ", sum(y_train)/len(y_train))
print("Validation: ", sum(y_val)/len(y_val))
print("Test: ", sum(y_test)/len(y_test))

Train:  [0.36998338 0.63001662]
Validation:  [0.36640424 0.63359576]
Test:  [0.36907706 0.63092294]


In [56]:
model = BiLSTM(embedding_matrix, vocab_size = len(windex) + 1)

In [57]:
model.train_model()

In [54]:
model.get_model_summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 30, 300)      32430300    input_11[0][0]                   
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 30, 300)      32430300    input_12[0][0]                   
____________________________________________________________________________________________

In [53]:
print(q1_val.shape)

(72783, 30)


In [60]:
model.model.fit([q1_train, q2_train], y_train, epochs = 5, validation_data = ([q1_val, q2_val], y_val), batch_size = 64, validation_batch_size=32, verbose = 1)

Epoch 1/5
4549/4549 [==============================] - 460s 101ms/step - loss: 0.6539 - accuracy: 0.6300 - val_loss: 0.6496 - val_accuracy: 0.6336
Epoch 2/5
4549/4549 [==============================] - 409s 90ms/step - loss: 0.6502 - accuracy: 0.6300 - val_loss: 0.6471 - val_accuracy: 0.6336
Epoch 3/5
4549/4549 [==============================] - 392s 86ms/step - loss: 0.6479 - accuracy: 0.6300 - val_loss: 0.6447 - val_accuracy: 0.6336
Epoch 4/5
4549/4549 [==============================] - 379s 83ms/step - loss: 0.6465 - accuracy: 0.6300 - val_loss: 0.6446 - val_accuracy: 0.6336
Epoch 5/5
4549/4549 [==============================] - 367s 81ms/step - loss: 0.6458 - accuracy: 0.6299 - val_loss: 0.6429 - val_accuracy: 0.6336


In [61]:
y_pred = model.predict([q1_test, q2_test])
y_pred1d, y_actual1d = [], []
for i in range(len(y_test)):
    if(y_test[i][0] == 1):
        y_actual1d.append(0)
    else:
        y_actual1d.append(1)

for i in range(len(y_pred)):
    if(y_pred[i][0] > y_pred[i][1]):
        y_pred1d.append(0)
    else:
        y_pred1d.append(1)

from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: ", accuracy_score(y_actual1d, y_pred1d))
print("F1 Score: ", f1_score(y_actual1d, y_pred1d))

Accuracy:  0.630922939954496
F1 Score:  0.7737004912961728
